<a href="https://colab.research.google.com/github/inoyamanaka/Pemrosesan_Text/blob/main/Tf-IDF%20n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [358]:
import pandas as pd
import tweepy
import csv
import re
from google.colab import files
!pip install git+https://github.com/tweepy/tweepy.git

  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-7e66fr46
  Running command git clone -q https://github.com/tweepy/tweepy.git /tmp/pip-req-build-7e66fr46


In [359]:
api_key = "wjfC7sqEsgkkOtAh3QUDKTTGJ"
api_secret = "TAwEmJ1MhYMgKwJBngh4ZpXnWdNFbdj778fiq6UkLFuLZ5ExKQ"
access_token = "1198104989102567424-noL01z6l2PvZvtfgwhrebESA3dmwN9"
access_secret = "gFTvVQoIO4J7gZ2yqVf5It6GaZmbtCz61FQphSfVJIHTJ"

auth = tweepy.OAuthHandler(api_key,api_secret)
auth.set_access_token(access_token,access_secret)

api = tweepy.API(auth,wait_on_rate_limit=True)
api.verify_credentials()

User(_api=<tweepy.api.API object at 0x7f8d56783390>, _json={'id': 1198104989102567424, 'id_str': '1198104989102567424', 'name': 'Herlambang Kurniawan', 'screen_name': 'ApiPrajurit', 'location': 'no where', 'description': 'Software engineer jadi2an', 'url': 'https://t.co/DqqAw5tVY6', 'entities': {'url': {'urls': [{'url': 'https://t.co/DqqAw5tVY6', 'expanded_url': 'http://Darkwebb.co.id', 'display_url': 'Darkwebb.co.id', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 2, 'friends_count': 35, 'listed_count': 2, 'created_at': 'Sat Nov 23 05:04:48 +0000 2019', 'favourites_count': 54, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 76, 'lang': None, 'status': {'created_at': 'Fri Mar 04 03:48:14 +0000 2022', 'id': 1499592527527038976, 'id_str': '1499592527527038976', 'text': '@tvOneNews HOAXXXX', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'tvOneN

In [11]:
output = []
for tweet in tweepy.Cursor(api.search_tweets, "Kkb Papua", count=50).items(50):
    user = tweet.user.screen_name
    tweet_id = tweet.id
    list_output = (tweet.text)
    output.append(list_output)

In [ ]:
outputs = pd.read_csv('output (10).csv', names=['Text'])
outputs

In [ ]:
df = pd.DataFrame(outputs,columns =['Text'])
df.head(20)

In [262]:
def preprocess(str_text):
  # 1. Casefolding - mengubah ke case yang sama semua (misal huruf kecil semua)
  str_text = str_text.lower()
  return str_text

df['Hasil Case Folding'] = df['Text'].apply(lambda x: preprocess(x))

In [ ]:
def preprocess(str_text):
  # 2. Hapus mention (@..) - menggunakan regular expression
  str_text = re.sub(r'@[\w]+',' ',str_text)
  # 3. Hapus hashtag (#..) - menggunakan regular expression
  str_text = re.sub(r'#[\w]+',' ',str_text)
  # Langkah 2 dan 3 bisa digabung dengan kode re: r'[@|#][\w]+'
  # 4. Hapus URL
  str_text = re.sub(r'http\S+',' ',str_text)
  # 5. Hapus non-printable chars, tanda baca, angka (alias hapus selain huruf)
  str_text = re.sub(r'[^a-zA-Z]+',' ',str_text)

  # Tokenize kalimat
  str_text = re.split("\s", str_text)
  return str_text

df['Hasil Tokenize'] = df['Hasil Case Folding'].apply(lambda x: preprocess(x))
df

In [ ]:
# Filtering words
ban_words = ['rt','dan', 'dari' ,'serta' ,'melainkan' ,'padahal' ,'sedangkan' ,'atau', 'tetapi',
             'jika','jikalau','bahwa','sehingga','sudah', 'yang','padahal', 'sedangkan', 'sambil',
             'supaya', 'agar', 'untuk','yg','aja','ini','daripada','ke','di','n','dengan','jadi',
             'd','adanya','saat','bahkan','sangat','dah','gw','doang','aja','ini','hanya','dalam',
             'tak','adalah','adanya','termasu','karena','mengapa', 'tidak', 'ada','b','t','g','p',
              'hrus', 'bisa','dob','bukan','dalam','malah','akibat','pem','md','lagi','tapi','sono',
             'sana','drun','bacot', 'p','pua','terbaikdi','drun','gorok','oap','pua', 'mbak', 'kampung',
             'tumb', 'lkan','ny','usah','ntai','roris','ring','artinya','juga','ara','dong','yaa','cuma',
             'gore','akan','maybrat','oleh','sugapa','k','hari','akh','mengut','sebenarnya','pa',
             'begitu','kira','kurang','lebihnya','bro','jelas','malah','aneh','anies','saatnya',
             'terima','kasih','ammi','gat','hendrik']


def filter2(str_text):

  # Hapus kata ban words
  for i in str_text:
    if i in ban_words:
      str_text.remove(i)

  return str_text
    
# df['Hasil Filtering'] = df['Hasil Tokenize'].apply(lambda x: filter1(x))
df['Hasil Filtering'] = df['Hasil Tokenize'].apply(lambda x: filter2(x))
df[['Hasil Filtering']]

In [360]:
# STEMMING
# kamus normalisasi
norm = {'pembentukan':'bentuk','pendidikan':'didik','bukanlah':'bukan','pedalaman':'dalam','terdepan':'depan',
        'kesejahteraan':'sejahtera','terwujud':'wujud','pemekaran':'mekar','dianggap':'anggap','lihatlah':'lihat',
        'dihentikan':'henti','menghambat':'hambat','pembangunan':'bangun','berdemo':'demo','menolak':'tolak',
        'pendukung':'dukung','menebarkan':'tebar','mengumumkan':'umum','nguasain':'kuasa','terbaik':'baik','pekerja':'kerja',
        'tersangka':'sangka','merajalela':'rajalela','melakukan':'laku','menghambat':'hambat','kemajuan':'maju','memasok':'pasok',
        'mengungsi':'ungsi','penyerangan':'serang','mengakibatkan':'akibat','ditembak':'tembak','berhari':'hari','penanganan':'angan',
        'menggangap':'anggap','menghalangi':'alang','tersangka':'sangka','terduga':'duga','membangun':'bangun','menganggap':'anggap',
        'mendukung':'dukung','memajukan':'maju','serangan':'serang','pendekatan':'dekat'
        }

In [ ]:
def preprocess(str_text):
  replacer = norm.get  
  return [replacer(n, n) for n in str_text]


df['Hasil Stemming'] = df['Hasil Filtering'].apply(lambda x: preprocess(x))
df[['Hasil Stemming']]

In [223]:
df[['Text','Hasil Case Folding']].to_excel("compact.xlsx")

In [366]:
df[['Hasil Case Folding','Hasil Tokenize']].to_excel("compact1.xlsx")
df[['Hasil Tokenize','Hasil Filtering']].to_excel("compact2.xlsx")
df[['Hasil Filtering','Hasil Stemming']].to_excel("compact3.xlsx")

In [267]:
pemrograman1 = []
def gabung(str_text):
  pemrograman1.extend(str_text)

  return pemrograman1
    
baru = df['Hasil Stemming'].apply(lambda x: gabung(x))


In [ ]:
baru

In [326]:
bar = baru[0]

def my_function(x):
  return list(dict.fromkeys(x))


In [ ]:
bar

In [270]:
bar = my_function(bar)
bar.remove('')

In [ ]:
bar

In [271]:
word_set = bar

In [ ]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

pd.set_option('display.max_rows', None)

In [356]:
sentences = ['dukung', 'bentuk', 'provinsi', 'papua', 'barat', 'daya','papua']
pd.set_option('max_rows', 99999)

def cnt(str_text):
  word_count = {}
  jumlah = []
  for word in (word_set):
      word_count[word] = 0
      for sent in str_text:
          if word in sent:
              word_count[word] += 1

  # Mencetak isi dictionary dengan perulangan
  for key in word_count:
    jumlah.append(word_count[key])

  return jumlah


def gabung2(str_text):
  pemrograman = []
  pemrograman.extend(str_text)

  return pemrograman
    
df_count = pd.DataFrame(word_set, columns =['Kata'])
we_n = df['Hasil Stemming'].apply(lambda x: gabung2(x))


for i in range (1,51):
  print(we_n[i-1])
  df_count[f'D{i}'] = cnt(we_n[i-1])

df_count


['dukung', 'bentuk', 'provinsi', 'papua', 'barat', 'daya', '']
['kstp', 'bukan', 'representasi', 'masyarakat', 'papua', '']
['satgas', 'tni', 'garda', 'depan', 'didik', 'dalam', 'papua', '']
['sejahtera', 'papua', 'wujud', 'adanya', 'otsus', 'mekar', '']
['', 'kkb', 'papua', 'kebal', 'hukum', '']
['sniper', 'baik', 'pasukan', 'dunia', 'ini', 'teroris', 'kkb', 'papua', '']
['baru', 'duga', 'teroris', 'langsung', 'tembak', 'mati', 'jelas', 'teroris', 'kkb', 'papua', 'anggap', 'saudara', '']
['', 'lihat', 'dari', 'teroris', 'kkb', 'segala', 'aktivitas', 'henti', 'bahkan', 'hambat', 'bangun', 'papua']
['', 'lihat', 'dari', 'teroris', 'kkb', 'segala', 'aktivitas', 'henti', 'bahkan', 'hambat', 'bangun', 'papua']
['', 'lihat', 'dari', 'teroris', 'kkb', 'segala', 'aktivitas', 'henti', 'bahkan', 'hambat', 'bangun', 'papua']
['baru', 'duga', 'teroris', 'langsung', 'tembak', 'mati', 'jelas', 'teroris', 'kkb', 'papua', 'anggap', 'saudara', '']
['', 'dukung', 'teroris', 'kkb', 'tolak', 'mekar', 'de

,Kata,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D41,D42,D43,D44,D45,D46,D47,D48,D49,D50
0,dukung,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,bentuk,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,provinsi,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,papua,1,1,1,1,1,1,1,1,1,...,1,2,1,1,1,1,0,1,2,2
4,barat,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,daya,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,kstp,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,bukan,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,representasi,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,masyarakat,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [357]:
df_count.to_excel("final.xlsx")

In [ ]:
def frequencyTerm(text):
  data = []
  dict_of_counts = {item:text.count(item) for item in text}
  for y in dict_of_counts:
    # data.append("('"+y+"',"+str(dict_of_counts[y])+"),")
    # data.append(dict_of_counts[y]) #excel
    data.append((str(y), dict_of_counts[y]))
  print(data)
# frequencyTerm(['honor', 'rosa', 'painter', 'doodle', 'birthday'])
[frequencyTerm(x) for x in df['Hasil Stemming']]